In [1]:

import os
import sys
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DateType,DecimalType,DoubleType,BooleanType,TimestampType
from pyspark.shell import spark

from pyspark.sql.functions import initcap, lower,col,year,month,dayofmonth,expr
from pyspark.sql import functions as sf
from pyspark.sql.functions import regexp_replace
import datetime
import pandas as pd
import numpy as np
 

#import org.apache.spark.sql.functions.format_number




os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
spark_app = SparkSession.builder.config("spark.driver.host", "localhost").getOrCreate()









 




Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.0
      /_/

Using Python version 3.9.12 (main, Apr  4 2022 05:22:27)
Spark context Web UI available at http://LAPTOP-PSIMHJKQ:4043
Spark context available as 'sc' (master = local[*], app id = local-1661364850251).
SparkSession available as 'spark'.


# Functional Requirement 1.1
1. Functional Requirements - Load Credit Card Database - cdw_sapp_custmer (SQL)


In [2]:
# cdw_sapp_custmer details

#df_cust = spark.read.schema(schema).json('C:/Users/asha_/dev/capstoneproj/Data/cdw_sapp_custmer.json')
df_cust = spark.read.format('org.apache.spark.sql.json').load('C:/Users/asha_/dev/capstoneproj/Data/cdw_sapp_custmer.json')
#df_cust.show(5)
#df_cust.printSchema()






In [7]:
#custmer table
# column merged from street_name,apt no to full_street_address
df_cust1 = df_cust.withColumn("FULL_STREET_ADDRESS", expr(" APT_NO ||','|| STREET_NAME"))
df_cust1 = df_cust1.drop('APT_NO')
df_cust1 = df_cust1.drop('street_name')
df_cust1.printSchema()




root
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: long (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- FULL_STREET_ADDRESS: string (nullable = true)



In [8]:
#changing datatype in custmer table
df_cust1 = df_cust1.withColumn("CUST_PHONE",col("CUST_PHONE").cast("string"))
df_cust1 = df_cust1.withColumn("SSN",col("SSN").cast("int"))
df_cust1 = df_cust1.withColumn("CUST_ZIP",col("CUST_ZIP").cast("int"))
df_cust = df_cust1.withColumn("LAST_UPDATED",col("LAST_UPDATED").cast("timestamp"))
df_cust1.printSchema()

root
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: string (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: integer (nullable = true)
 |-- FULL_STREET_ADDRESS: string (nullable = true)



In [9]:
# changing first_name,last_name and middle_name columns
df_cust1 = df_cust1.withColumn('MIDDLE_NAME', sf.lower(sf.col('MIDDLE_NAME')))
df_cust1 = df_cust1.withColumn('FIRST_NAME', sf.initcap(sf.col('FIRST_NAME')))
df_cust1 = df_cust1.withColumn('LAST_NAME', sf.initcap(sf.col('LAST_NAME')))
df_cust1.select('first_name','last_name','middle_name').show()


+----------+---------+-----------+
|first_name|last_name|middle_name|
+----------+---------+-----------+
|      Alec|   Hooper|         wm|
|      Etta|   Holman|    brendan|
|    Wilber|   Dunham|   ezequiel|
|   Eugenio|    Hardy|      trina|
|   Wilfred|    Ayers|        may|
|      Beau|  Woodard|    ambrose|
|    Sheila|     Kemp|      larry|
|     Wendy|   Hurley|        ora|
|      Alec|  Gilmore|     tracie|
|    Barbra|      Lau|    mitchel|
|   Edmundo|  Thomson|      denny|
|      Elsa|   Truong|   isabelle|
|     Homer| Mckinney|      henry|
|      Rita|     Kidd|     rickey|
|    Amalia|  Ballard|  heriberto|
|     Patty|   Thomas|   angelita|
|  Josefina|   Morrow|   dorothea|
|    Nelson|  Andrews|  jefferson|
|    Miquel|Schneider|     maximo|
|    Parker|  Tidwell|    arnulfo|
+----------+---------+-----------+
only showing top 20 rows



In [10]:
#changing ph no format
df_cust1 = df_cust1.withColumn("CUST_PHONE",regexp_replace('CUST_PHONE',r'(\d{3})(\d{4})',r'(999)$1-$2'))
df_cust1.show(2)


+----------------+------------+-------------+-------------------+-------------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|   CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN| FULL_STREET_ADDRESS|
+----------------+------------+-------------+-------------------+-------------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|4210653310061055|     Natchez|United States|AHooper@example.com|(999)123-7818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         wm|123456100|656,Main Street N...|
|4210653310102868|Wethersfield|United States|EHolman@example.com|(999)123-8933|        CT|    6109|      Etta|   Holman|2018-04-21T12:49:...|    brendan|123453023|   829,Redwood Drive|
+----------------+------------+-------------+-------------------+----------

# Function Requirement 1.2

Data loading into Database

In [11]:
df_cust1.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone_") \
.option("dbtable", "creditcard_capstone_.CDW_SAPP_CUSTOMER") \
.option("user", "root") \
.option("password", "a") \
.save()